In [2]:
import numpy
import os
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from python_speech_features import mfcc
from python_speech_features.base import delta
import warnings
from sklearn import preprocessing
warnings.filterwarnings('ignore')
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [7]:
mfcc_features = []
delta_features = []
english_features = []
num_iterations = 0
for file in os.listdir("C:\\Users\Lenovo\Desktop\\Utility\Semester 5\Machine Learning\Project\English_Data\\"):
    data, samplerate = sf.read("C:\\Users\Lenovo\Desktop\\Utility\Semester 5\Machine Learning\Project\English_Data\\" + file)
    hamming_window = numpy.hamming(400)
    temp_mfcc_features = []
    for i in range(0,len(data)-400,240):
        trimmed = data[i:i+400]
        hammed = numpy.multiply(hamming_window, trimmed)
        mfcced = mfcc(hammed,samplerate)
        mfcc_features.append(mfcced[0])
        temp_mfcc_features.append(mfcced[0])
        num_iterations+=1
#     hamming_window = numpy.hamming(len(data)-(i+240))
#     trimmed = data[i+240:len(data)]
#     hammed = (numpy.multiply(hamming_window, trimmed))
#     mfcced = mfcc(hammed,samplerate)
#     mfcc_features.append(mfcced[0])
#     temp_mfcc_features.append(mfcced[0])
#     mfcc_features = numpy.array(mfcc_features)
    print(delta(temp_mfcc_features, 1).shape)
    delta_features.extend(delta(temp_mfcc_features, 1))
    break
    if (num_iterations>=5000):
        break
print ("Done")

(389, 13)
Done


In [3]:
hindi_mfcc_features = []
hindi_delta_features = []
hindi_features = []
num_iterations = 0
for file in os.listdir("C:\\Users\Lenovo\Desktop\\Utility\Semester 5\Machine Learning\Project\Hindi_Data_flac\\"):
    data, samplerate = sf.read("C:\\Users\Lenovo\Desktop\\Utility\Semester 5\Machine Learning\Project\Hindi_Data_flac\\" + file)
    hamming_window = numpy.hamming(400)
    data = data[:,0]
    temp_mfcc_features = []
    print (data)
    for i in range(0,len(data)-400,240):
        trimmed = data[i:i+400]
        hammed = numpy.multiply(hamming_window, trimmed)
        mfcced = mfcc(hammed,samplerate,nfft=2048)
        hindi_mfcc_features.append(mfcced[0])
        temp_mfcc_features.append(mfcced[0])
        num_iterations+=1
#         break
#     break
#     hamming_window = numpy.hamming(len(data)-(i+240))
#     trimmed = data[i+240:len(data)]
#     hammed = (numpy.multiply(hamming_window, trimmed))
#     mfcced = mfcc(hammed,samplerate)
#     hindi_mfcc_features.append(mfcced[0])
#     temp_mfcc_features.append(mfcced[0])
#     mfcc_features = numpy.array(mfcc_features)
    hindi_delta_features.extend(delta(temp_mfcc_features, 1))
    if (num_iterations>=5000):
        break

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -6.10351562e-05
 -6.10351562e-05 -9.15527344e-05]
[ 0.          0.          0.         ... -0.00918579  0.02752686
  0.03726196]
[ 0.          0.          0.         ...  0.00012207 -0.00164795
 -0.0012207 ]


In [6]:
print(len(mfcc_features), len(delta_features))
if (len(hindi_mfcc_features) != len(hindi_delta_features)):
    print("PROBLEM")

5132 5132


In [7]:
zeros = [0]*len(mfcc_features)
mfcc_features = numpy.array(mfcc_features)
delta_features = numpy.array(delta_features)
x_English = numpy.concatenate((mfcc_features,delta_features),axis = 1)

In [8]:
ones = [1]*len(hindi_mfcc_features)
hindi_mfcc_features = numpy.array(hindi_mfcc_features)
hindi_delta_features = numpy.array(hindi_delta_features)
x_Hindi = numpy.concatenate((hindi_mfcc_features, hindi_delta_features),axis = 1)

In [9]:
X = numpy.concatenate((x_English, x_Hindi), axis=0)
Y = numpy.concatenate((zeros, ones), axis=0)
normalized_X = preprocessing.normalize(X)
x_train, x_test, y_train, y_test = train_test_split(normalized_X, Y, test_size = 0.2)
# parameters = [{'kernel': ['rbf'], 'gamma' : [0.001,0.05,0.1,10], 'C' : [0.001, 0.01, 0.1, 1,2.5,100]}]
# clf = GridSearchCV(svm.SVC(), parameters, cv = 5)
# clf = GridSearchCV(svm.SVC())
clf = svm.SVC()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
unique, counts = numpy.unique(predictions, return_counts=True)
print(dict(zip(unique, counts)), "hi")
# print(predictions.count(1))
print(sum(predictions),len(predictions))
print(accuracy_score(predictions, y_test))

{0: 1018, 1: 1109} hi
1109 2127
0.9717912552891397


In [113]:
mfcc_features = []
delta_features = []
test_English = []
data, samplerate = sf.read("C:\\Users\Lenovo\Desktop\\Utility\Semester 5\Machine Learning\Project\Sample_English.flac")
print(data.shape)
data = data[:,0]
num_iterations = 0
hamming_window = numpy.hamming(400)
temp_mfcc_features = []
for i in range(0,len(data)-400,240):
    trimmed = data[i:i+400]
    hammed = numpy.multiply(hamming_window, trimmed)
    mfcced = mfcc(hammed, samplerate, nfft = 2048)
    mfcc_features.append(mfcced[0])
    temp_mfcc_features.append(mfcced[0])
    num_iterations+=1
    if (num_iterations>=1000):
        break
delta_features.extend(delta(temp_mfcc_features, 1))


test_English = numpy.concatenate((mfcc_features,delta_features),axis = 1)


(6478848, 2)


In [114]:
test_English = preprocessing.normalize(test_English)
results = clf.predict(test_English)
print(results)
print(sum(results))
print (1-sum(results)/len(test_English))

[1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1
 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [115]:
mfcc_features = []
delta_features = []
test_English = []
data, samplerate = sf.read("C:\\Users\Lenovo\Desktop\\Utility\Semester 5\Machine Learning\Project\Sample_Hindi.flac")
print(data.shape)
data = data[:,0]
num_iterations = 0
hamming_window = numpy.hamming(400)
temp_mfcc_features = []
for i in range(0,len(data)-400,240):
    trimmed = data[i:i+400]
    hammed = numpy.multiply(hamming_window, trimmed)
    mfcced = mfcc(hammed, samplerate, nfft = 2048)
    mfcc_features.append(mfcced[0])
    temp_mfcc_features.append(mfcced[0])
    num_iterations+=1
    if (num_iterations>=1000):
        break
delta_features.extend(delta(temp_mfcc_features, 1))


test_English = numpy.concatenate((mfcc_features,delta_features),axis = 1)


(5870592, 2)


In [ ]:
test_English = preprocessing.normalize(test_English)
results = clf.predict(test_English)
print(results)
print(sum(results))
print (sum(results)/len(results))